In [1]:
"""
Preprocessing utilities for the Video Game Success Prediction project.

This module provides functions for cleaning the dataset, including:
- Removing duplicate rows
- Converting a release date column into a year
- Imputing missing values by dtype
- Only dropping rows where total_sales is missing
"""

from __future__ import annotations

from typing import Optional, Sequence

import pandas as pd


def clean_dataset(
	df: pd.DataFrame,
	*,
	date_col: str = "release_date",
	fill_unknown: str = "Unknown",
	bool_fill: bool = False,
) -> pd.DataFrame:
	"""
	Clean a video game sales dataframe.

	Operations performed (in order):
	1) Drop exact duplicate rows.
	2) Strip leading/trailing whitespace in object columns.
	3) Convert `date_col` to year (Int64) if present, coercing invalids to <NA>.
	4) Drop rows with missing `total_sales` only.
	5) Impute missing values by dtype:
	   - Numeric (including pandas nullable ints): median
	   - Categorical/object/category: fill with `fill_unknown`
	   - Boolean/bool/boolean: fill with `bool_fill`

	Parameters
	----------
	df : pd.DataFrame
		Input dataframe.
	date_col : str, default "release_date"
		Column name to parse as dates and convert to year.
	fill_unknown : str, default "Unknown"
		Token used to fill missing values for categorical columns.
	bool_fill : bool, default False
		Value used to fill missing values for boolean columns.

	Returns
	-------
	pd.DataFrame
		A cleaned copy of the input dataframe.

	Notes
	-----
	- The `date_col` is replaced in-place with the extracted year as pandas Int64
	  (nullable integer) to preserve missing years.
	- If `date_col` is not present, the step is skipped.
	- Only rows where `total_sales` is missing are dropped; no generic drop of all-NaN rows occurs.
	"""

	# Work on a copy to avoid mutating callers' data
	cleaned = df.copy()

	# 1) Remove exact duplicate rows
	cleaned = cleaned.drop_duplicates()

	# 2) Strip whitespace in object columns without converting NaN to strings
	obj_cols = cleaned.select_dtypes(include=["object"]).columns
	if len(obj_cols) > 0:
		for c in obj_cols:
			cleaned[c] = cleaned[c].apply(lambda x: x.strip() if isinstance(x, str) else x)

	# 3) Convert release date to year (nullable Int64) if present
	if date_col in cleaned.columns:
		parsed = pd.to_datetime(cleaned[date_col], errors="coerce", utc=False)
		# Extract year; keep as nullable integer to preserve missing
		cleaned[date_col] = parsed.dt.year.astype("Int64")

	# 4) Drop rows with missing total_sales ONLY
	if "total_sales" in cleaned.columns:
		cleaned = cleaned[~cleaned["total_sales"].isna()].reset_index(drop=True)

	# 5) Impute missing values by dtype
	# Identify numeric columns including pandas nullable ints
	num_cols = list(cleaned.select_dtypes(include=["number"]).columns)
	# Include pandas nullable integer dtypes explicitly (Int64, Int32, Int16)
	nullable_int_cols = list(
		cleaned.select_dtypes(include=["Int64", "Int32", "Int16"]).columns
	)
	for c in set(nullable_int_cols) - set(num_cols):
		num_cols.append(c)

	# Boolean columns (both numpy bool and pandas BooleanDtype)
	bool_cols = list(cleaned.select_dtypes(include=["bool", "boolean"]).columns)

	# Categorical/object/category columns
	cat_cols = list(
		cleaned.select_dtypes(include=["object", "category"]).columns
	)

	# Remove overlaps to avoid double-filling
	cat_cols = [c for c in cat_cols if c not in num_cols and c not in bool_cols]

	# Fill numeric columns with median
	for c in num_cols:
		if c in cleaned.columns:
			median_val = cleaned[c].median()
			cleaned[c] = cleaned[c].fillna(median_val)

	# Fill boolean columns with specified bool_fill
	for c in bool_cols:
		if c in cleaned.columns:
			cleaned[c] = cleaned[c].fillna(bool_fill)

	# Fill categorical columns with fill_unknown token
	for c in cat_cols:
		if c in cleaned.columns:
			cleaned[c] = cleaned[c].fillna(fill_unknown)

	return cleaned



def add_hit_label(
	df: pd.DataFrame,
	*,
	sales_col: str = "total_sales",
	threshold: float = 1.0,
	label_col: str = "Hit",
	dtype: str = "Int8",
) -> pd.DataFrame:
	"""
	Add a binary hit label column based on a sales threshold.

	A value of 1 indicates `sales_col >= threshold`, else 0. Missing sales are
	treated as 0.

	Parameters
	----------
	df : pd.DataFrame
		Input dataframe.
	sales_col : str, default "total_sales"
		Column containing total sales values (numeric).
	threshold : float, default 1.0
		Threshold in the same units as `sales_col` to define a hit.
	label_col : str, default "Hit"
		Name of the output binary column.
	dtype : str, default "Int8"
		The dtype used for the binary column.

	Returns
	-------
	pd.DataFrame
		A copy of df with the new label column appended.
	"""
	out = df.copy()
	if sales_col not in out.columns:
		raise KeyError(f"Column '{sales_col}' not found in DataFrame")
	is_hit = out[sales_col].fillna(0).ge(threshold)
	out[label_col] = is_hit.astype(dtype)
	return out



def encode_categoricals(
	df: pd.DataFrame,
	*,
	columns: Optional[Sequence[str]] = ("genre", "platform", "publisher"),
	drop_first: bool = False,
	dummy_na: bool = False,
	prefix_sep: str = "=",
) -> pd.DataFrame:
	"""
	One-hot encode selected categorical columns using pandas.get_dummies.

	Parameters
	----------
	df : pd.DataFrame
		Input dataframe.
	columns : sequence of str, optional
		Categorical column names to encode. Defaults to ("genre","platform","publisher").
	drop_first : bool, default False
		Whether to drop the first category for each encoded variable.
	dummy_na : bool, default False
		Whether to add a column for NaNs.
	prefix_sep : str, default "="
		Separator between the column name and category in the dummy column names.

	Returns
	-------
	pd.DataFrame
		Dataframe with specified categorical columns one-hot encoded.
	"""
	if columns is None:
		columns = []
	existing = [c for c in columns if c in df.columns]
	if len(existing) == 0:
		# Nothing to encode; return a copy to avoid side effects
		return df.copy()
	encoded = pd.get_dummies(
		df,
		columns=list(existing),
		drop_first=drop_first,
		dummy_na=dummy_na,
		prefix=existing,
		prefix_sep=prefix_sep,
	)
	return encoded


__all__ = [
	"clean_dataset",
	"add_hit_label",
	"encode_categoricals",
]

In [2]:
from __future__ import annotations

import pandas as pd
from typing import Optional, Sequence


def handle_missing_values(df: pd.DataFrame) -> pd.DataFrame:
    """
    Impute missing values:
    - Numeric columns: median
    - Categorical/object/category/boolean columns: most frequent (mode)

    Returns a new DataFrame; original is not modified.
    """
    out = df.copy()

    # Numeric median imputation
    num_cols = out.select_dtypes(include=["number"]).columns.tolist()
    for c in num_cols:
        if c in out.columns:
            median_val = out[c].median()
            if pd.notna(median_val):
                out[c] = out[c].fillna(median_val)

    # Categorical/boolean mode imputation
    cat_cols = out.select_dtypes(include=["object", "category", "bool", "boolean"]).columns.tolist()
    for c in cat_cols:
        if c in out.columns:
            mode_vals = out[c].mode(dropna=True)
            if not mode_vals.empty:
                out[c] = out[c].fillna(mode_vals.iloc[0])

    return out


def clean_dataset(
    df: pd.DataFrame,
    *,
    date_col: str = "release_date",
    fill_unknown: str = "Unknown",
    bool_fill: bool = False,
) -> pd.DataFrame:
    """
    Clean a video game sales dataframe.

    Steps:
    1) Drop exact duplicate rows.
    2) Strip leading/trailing whitespace in object columns.
    3) Convert `date_col` to year (Int64) if present, coercing invalids to <NA>.
    4) Drop rows with missing `total_sales` only (do NOT drop all-NaN rows).
    5) Impute missing values by dtype:
       - Numeric (incl. nullable ints): median
       - Categorical/object/category: fill with `fill_unknown`
       - Boolean/bool/boolean: fill with `bool_fill`

    Notes
    -----
    - The `date_col` is replaced in-place with the extracted year as pandas Int64
      (nullable integer) to preserve missing years.
    - If `date_col` is not present, that step is skipped.
    - Row drop for missing `total_sales` happens before imputation.
    """
    cleaned = df.copy()

    # 1) Remove exact duplicate rows
    cleaned = cleaned.drop_duplicates()

    # 2) Strip whitespace in object columns without converting NaN to strings
    obj_cols = cleaned.select_dtypes(include=["object"]).columns
    if len(obj_cols) > 0:
        for c in obj_cols:
            cleaned[c] = cleaned[c].apply(lambda x: x.strip() if isinstance(x, str) else x)

    # 3) Convert release date to year (nullable Int64) if present
    if date_col in cleaned.columns:
        parsed = pd.to_datetime(cleaned[date_col], errors="coerce", utc=False)
        cleaned[date_col] = parsed.dt.year.astype("Int64")

    # 4) Drop rows with missing total_sales ONLY
    if "total_sales" in cleaned.columns:
        cleaned = cleaned[~cleaned["total_sales"].isna()].reset_index(drop=True)

    # 5) Impute missing values by dtype (retain prior behavior here)
    # Numeric columns including pandas nullable ints
    num_cols = list(cleaned.select_dtypes(include=["number"]).columns)
    nullable_int_cols = list(cleaned.select_dtypes(include=["Int64", "Int32", "Int16"]).columns)
    for c in set(nullable_int_cols) - set(num_cols):
        num_cols.append(c)

    # Boolean columns (numpy bool and pandas BooleanDtype)
    bool_cols = list(cleaned.select_dtypes(include=["bool", "boolean"]).columns)

    # Categorical/object/category columns
    cat_cols = list(cleaned.select_dtypes(include=["object", "category"]).columns)

    # Remove overlaps
    cat_cols = [c for c in cat_cols if c not in num_cols and c not in bool_cols]

    # Fill numeric with median
    for c in num_cols:
        if c in cleaned.columns:
            median_val = cleaned[c].median()
            cleaned[c] = cleaned[c].fillna(median_val)

    # Fill boolean with specified bool_fill
    for c in bool_cols:
        if c in cleaned.columns:
            cleaned[c] = cleaned[c].fillna(bool_fill)

    # Fill categorical with fill_unknown token
    for c in cat_cols:
        if c in cleaned.columns:
            cleaned[c] = cleaned[c].fillna(fill_unknown)

    return cleaned


# Update public API
__all__ = [
    "clean_dataset",
    "add_hit_label",
    "encode_categoricals",
    "handle_missing_values",
]

In [ ]:
# Quick sanity test: only drop rows with total_sales NaN
import pandas as pd
from pandas.testing import assert_frame_equal

from preprocessing import clean_dataset

# Construct a small test DataFrame
_df = pd.DataFrame({
    "name": ["A", None, None, "D"],
    "release_date": ["2010-01-01", None, None, "2012-05-03"],
    "total_sales": [1.2, None, 0.5, None],
    "genre": [" Action ", None, "RPG", " Sports"],
})

# Row-by-row expectations:
# 0: total_sales=1.2 -> keep
# 1: total_sales=None -> drop
# 2: total_sales=0.5 -> keep
# 3: total_sales=None -> drop

_cleaned = clean_dataset(_df)

# Ensure rows 1 and 3 dropped, but row 2 kept even if other fields are NaN or whitespace
assert len(_cleaned) == 2, f"Expected 2 rows after drop, got {len(_cleaned)}"
assert set(_cleaned.index) == {0, 1} or True  # index was reset; verify content instead

# Validate that whitespace was stripped and year extracted
expected_years = pd.Series([2010, pd.NA], dtype="Int64")
assert _cleaned["release_date"].dtype.name == "Int64"

# Check names trimmed where present
assert _cleaned.loc[0, "genre"] == "Action" or _cleaned.loc[0, "genre"] == "Action", "Genre should be trimmed"

print("Sanity test passed: only total_sales NaN rows are dropped.")

In [4]:
# How many rows remain after preprocessing?
import pandas as pd

# Load the raw/processed dataset path used by this project
# Adjust path if needed based on your workflow
csv_path = "../data/vg_sales_2024.csv"

df_raw = pd.read_csv(csv_path)
print(f"Rows before preprocessing: {len(df_raw):,}")

# Use the clean_dataset defined above in this notebook
# (no import needed since it's already defined in earlier cells)
df_clean = clean_dataset(df_raw)
print(f"Rows after preprocessing:  {len(df_clean):,}")

Rows before preprocessing: 64,016
Rows after preprocessing:  18,922


In [6]:
# Compute final train/test sizes using current preprocessing
from sklearn.model_selection import train_test_split

# df_clean is already available from the previous cell
# Create Hit label consistent with training using the function defined above in this notebook

df_labeled = add_hit_label(df_clean, sales_col='total_sales', threshold=1.0, label_col='Hit')

# Features used in training
feature_cols_cat = ['genre', 'platform', 'publisher']
feature_cols_num = ['critic_score', 'release_year']

# Ensure columns exist
for col in feature_cols_cat + feature_cols_num:
    if col not in df_labeled.columns:
        df_labeled[col] = pd.NA

X = df_labeled[feature_cols_cat + feature_cols_num].copy()
y = df_labeled['Hit'].astype('Int64').fillna(0).astype(int)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Train samples: {len(X_train):,}")
print(f"Test samples:  {len(X_test):,}")

Train samples: 15,137
Test samples:  3,785


In [8]:
# Evaluate saved models on the same test split
import os
import pickle
from pathlib import Path
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# Resolve project root relative to this notebook's location
nb_path = Path(os.getcwd())
project_root = nb_path
if (nb_path / 'src').exists() and (nb_path / 'data').exists():
    project_root = nb_path
elif (nb_path.parent / 'data').exists():
    project_root = nb_path.parent

# Helper to evaluate a model expecting the same features

def _evaluate_model(model, X_test, y_test, name: str):
    try:
        y_pred = model.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        prec = precision_score(y_test, y_pred, zero_division=0)
        rec = recall_score(y_test, y_pred, zero_division=0)
        f1 = f1_score(y_test, y_pred, zero_division=0)
        print(f"{name}: acc={acc:.3f}, prec={prec:.3f}, rec={rec:.3f}, f1={f1:.3f}")
    except Exception as e:
        print(f"{name}: failed to evaluate -> {e}")

# Load each available model and evaluate
candidates = [
    (project_root / 'model.pkl', 'model.pkl'),
    (project_root / 'model_rf.pkl', 'model_rf.pkl'),
    (project_root / 'model_lr.pkl', 'model_lr.pkl'),
    (project_root / 'model_xgb.pkl', 'model_xgb.pkl'),
]

for path, label in candidates:
    if path.exists():
        try:
            with open(path, 'rb') as f:
                mdl = pickle.load(f)
            _evaluate_model(mdl, X_test, y_test, label)
        except Exception as e:
            print(f"{label}: load failed -> {e}")
    else:
        print(f"{label}: not found")

C:\Users\USER\AppData\Roaming\Python\Python313\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator OneHotEncoder from version 1.4.2 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\USER\AppData\Roaming\Python\Python313\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator FunctionTransformer from version 1.4.2 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\USER\AppData\Roaming\Python\Python313\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator ColumnTransformer from versio

model.pkl: failed to evaluate -> float() argument must be a string or a real number, not 'NAType'


C:\Users\USER\AppData\Roaming\Python\Python313\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.4.2 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\USER\AppData\Roaming\Python\Python313\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.4.2 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\USER\AppData\Roaming\Python\Python313\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator Pipeline from ver

model_rf.pkl: failed to evaluate -> float() argument must be a string or a real number, not 'NAType'
model_lr.pkl: failed to evaluate -> float() argument must be a string or a real number, not 'NAType'
model_xgb.pkl: failed to evaluate -> float() argument must be a string or a real number, not 'NAType'


C:\Users\USER\AppData\Roaming\Python\Python313\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator OneHotEncoder from version 1.4.2 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\USER\AppData\Roaming\Python\Python313\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator FunctionTransformer from version 1.4.2 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\USER\AppData\Roaming\Python\Python313\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator ColumnTransformer from versio